In [1]:
from algo_gen import algo_genetique
from algo_four import algo_fourmis
import numpy as np
from random import randint

In [2]:
# Chargement de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(2, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat
 

In [3]:
algo_fourmis(create_matrix(10), verbal=True)

Matrice d'adjacence:
[[ 0.  1.  4.  6.  8.  8.  7. 10.  5.  1.]
 [ 1.  0.  1. 10.  5.  9.  9.  9.  7.  6.]
 [ 4.  1.  0.  1.  9.  5.  3.  9.  2.  6.]
 [ 6. 10.  1.  0.  1.  3.  8.  4.  5.  7.]
 [ 8.  5.  9.  1.  0.  1.  6.  7.  8.  2.]
 [ 8.  9.  5.  3.  1.  0.  1.  8.  3.  8.]
 [ 7.  9.  3.  8.  6.  1.  0.  1.  2.  4.]
 [10.  9.  9.  4.  7.  8.  1.  0.  1.  4.]
 [ 5.  7.  2.  5.  8.  3.  2.  1.  0.  1.]
 [ 1.  6.  6.  7.  2.  8.  4.  4.  1.  0.]]
iteration:  0 , meilleurs chemins: 14.0
iteration:  1 , meilleurs chemins: 10.0
iteration:  2 , meilleurs chemins: 10.0
iteration:  3 , meilleurs chemins: 10.0
iteration:  4 , meilleurs chemins: 10.0
iteration:  5 , meilleurs chemins: 10.0
iteration:  6 , meilleurs chemins: 10.0
iteration:  7 , meilleurs chemins: 10.0
iteration:  8 , meilleurs chemins: 10.0
iteration:  9 , meilleurs chemins: 10.0
iteration:  10 , meilleurs chemins: 10.0
iteration:  11 , meilleurs chemins: 10.0
iteration:  12 , meilleurs chemins: 10.0
iteration:  13 , meilleur

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10.0)

In [4]:
algo_genetique(create_matrix(7), time_max=100, verbal=True)

[[ 0.  1.  9.  8.  5.  7.  1.]
 [ 1.  0.  1.  3.  5.  7. 10.]
 [ 9.  1.  0.  1.  4.  7.  7.]
 [ 8.  3.  1.  0.  1.  4.  6.]
 [ 5.  5.  4.  1.  0.  1.  8.]
 [ 7.  7.  7.  4.  1.  0.  1.]
 [ 1. 10.  7.  6.  8.  1.  0.]]
iteration: 1 , nb individu: 20 val_min : 18.0
iteration: 2 , nb individu: 19 val_min : 19.0
iteration: 3 , nb individu: 15 val_min : 17.0
iteration: 4 , nb individu: 20 val_min : 17.0
iteration: 5 , nb individu: 13 val_min : 17.0
iteration: 6 , nb individu: 18 val_min : 20.0
iteration: 7 , nb individu: 17 val_min : 20.0
iteration: 8 , nb individu: 18 val_min : 20.0
iteration: 9 , nb individu: 13 val_min : 17.0
iteration: 10 , nb individu: 20 val_min : 13.0
iteration: 11 , nb individu: 20 val_min : 13.0
iteration: 12 , nb individu: 20 val_min : 13.0
iteration: 13 , nb individu: 20 val_min : 13.0
iteration: 14 , nb individu: 10 val_min : 18.0
iteration: 15 , nb individu: 20 val_min : 18.0
iteration: 16 , nb individu: 19 val_min : 18.0
iteration: 17 , nb individu: 18 val_min

([4, 5, 6, 0, 1, 2, 3], 7.0)